# 📓 LangGraph Quickstart with Otel

In this quickstart you will create a simple react agent with LangGraph and learn how to log it and get feedback on an LLM response.

For evaluation, we will leverage the RAG triad of groundedness, context relevance and answer relevance.

## Setup
### Add API keys
For this quickstart you will need an Open AI key

In [ ]:
# !pip install trulens langgraph trulens-providers-openai openai

In [ ]:
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "sk-proj-..."

os.environ["TRULENS_OTEL_TRACING"] = "1"

### Import from TruLens

In [ ]:
from trulens.core import TruSession

session = TruSession()
session.reset_database()

### Create an Agent

In [ ]:
from langgraph.prebuilt import create_react_agent
from trulens.core.otel.instrument import instrument
from trulens.otel.semconv.trace import SpanAttributes


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"


agent = create_react_agent(
    model="openai:gpt-4o",
    tools=[get_weather],
    prompt="You are a helpful assistant",
)


class WeatherAgent:
    def __init__(self, agent):
        self.agent = agent

    @instrument(
        span_type=SpanAttributes.SpanType.RECORD_ROOT,
        attributes={
            SpanAttributes.RECORD_ROOT.INPUT: "prompt",
            SpanAttributes.RECORD_ROOT.OUTPUT: "return",
        },
    )
    def invoke(self, prompt: str) -> str:
        """Invoke the agent with the given input data."""
        input_data = {"messages": [{"role": "user", "content": prompt}]}
        return self.agent.invoke(input_data)


weather_agent = WeatherAgent(agent)

## Initialize Feedback Function(s)

In [ ]:
from trulens.core import Feedback
from trulens.providers.openai import OpenAI

provider = OpenAI(model_engine="gpt-4.1-mini")

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input()
    .on_output()
)

## Instrument chain for logging with TruLens

In [ ]:
from trulens.apps.app import TruApp

tru_recorder = TruApp(
    weather_agent,
    app_name="ChatApplication",
    app_version="Base",
    feedbacks=[f_answer_relevance],
)

In [ ]:
with tru_recorder as recording:
    llm_response = weather_agent.invoke(
        "What's the weather like in San Francisco?"
    )

display(llm_response)

Check results

In [ ]:
session.get_leaderboard()

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(session=session)